## import libraries

In [3]:
import sys
!pip install torch torchvision torchtext pytorch_lightning tensorboard matplotlib tqdm wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.7/798.7 kB 9.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 14.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 5.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.3/596.3 kB 2.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 935.4 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 29.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.7/232.7 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.0/148.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 2.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 15.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=56a8c6d51921e7d8a6390ea9ffdc851b043a9057ae4977d1532d6c96cdb114d3
  Stored in directory: /home/atexier/.cache/pip/wheels/83/21/65/2ac62db55efa6e6edfad09f4e315aa82a35ab138f51e784fb1
Successfully built fire


In [1]:
import torch
torch.cuda.is_available()

True

In [17]:
import matplotlib.pyplot as plt
import torchvision.transforms as T
import numpy as np 
import csv
import os
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from torch import nn



## Load data

In [11]:
import urllib.request
import os
import zipfile
if not os.path.exists(os.getcwd()+"/ipeo_data"):
    with zipfile.ZipFile(os.getcwd()+"/ipeo_data.zip", 'r') as zip_ref:
        zip_ref.extractall()


In [44]:
val_csv = open(os.getcwd()+"/ipeo_data/splits/val.csv")
val_csv  = csv.reader(data_csv)
test_csv = open(os.getcwd()+"/ipeo_data/splits/test.csv")
test_csv  = csv.reader(test_csv)
val_csv = open(os.getcwd()+"/ipeo_data/splits/val.csv")
val_csv  = csv.reader(val_csv)

TypeError: object of type '_csv.reader' has no len()

## Set up the data

In [40]:
from torch.utils.data import Dataset

from PIL import Image

import os
import glob

"""CLASS rocks (1), scree (2), sparse rocks (3), water (4), glacier and permanent snow (5), forest(6), sparse forest(7),
grasslands and others (8)."""

class Alpine(Dataset):

    # mapping between label class names and indices
    LABEL_CLASSES = {
      'rocks': 		  1,
      'scree': 			    2,
      'sparse_rocks': 	  3,
      'water': 				      4,
      'glacier_and_permanent_snow': 			    5,
      'forest': 			    6,
      'sparse_forest':   7,
      'grasslands_and_others': 				    8,
      
    }

    def __init__(self, transforms=None, split='train'):
        self.transforms = transforms
        #only for label_image
        transforms_label = T.Compose([
        T.ToTensor()
        ])
        self.transforms_label = transforms_label
        
        # prepare data
        self.data = []                                  # list of tuples of (image path, label class)
        # get images with correct index according to dataset split
        if split=='train':
            data_csv = open(os.getcwd()+"/ipeo_data/splits/train.csv")
            data_csv  = csv.reader(data_csv)
        if split=='test':
            data_csv = open(os.getcwd()+"/ipeo_data/splits/test.csv")
            data_csv  = csv.reader(data_csv)
        if split=='val':
            data_csv = open(os.getcwd()+"/ipeo_data/splits/val.csv")
            data_csv  = csv.reader(data_csv)
            
        for row in data_csv :
            img_name = os.getcwd()+"/ipeo_data/rgb/"+row[0]+"_rgb.tif"
            img_label_name = os.getcwd()+"/ipeo_data/alpine_label/"+row[0]+"_label.tif"   
        # example format: 'baseFolder/agricultural/agricultural07.tif'
            self.data.append((
                img_name,
                img_label_name          # get index for label class
            ))


    #TODO: please provide the remaining functions required for the torch.utils.data.Dataset class.
    def __len__(self):
        return len(self.data)


    def __getitem__(self, x):
        img_name, img_label_name = self.data[x]
    
        img = Image.open(img_name)
        img_label = Image.open(img_label_name)
        
        
        if self.transforms is not None:
            img = self.transforms(img)
            img_label = self.transforms_label(img_label)
        return img,img_label


In [30]:
# initialize the dataset (call the constructor __init__)

dataset = Alpine()
print(f"dataset of length {len(dataset)}")
# plot individual samples
from ipywidgets import interact
@interact(idx=range(100))
def plot_sample(idx=0):
    img, img_label = dataset[idx]
    plt.figure(figsize=(15, 6))
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.subplot(1,2,2)
    plt.imshow(img_label)




dataset of length 7357


interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

### Find the mean and stf of the Dataset

In [48]:
train_dataset = Alpine(transforms=transforms_train, split='train')
train_dataset[0][0].shape

torch.Size([3, 400, 400])

In [41]:
transforms_train = T.Compose([
  T.ToTensor()
])

# dataset
train_dataset = Alpine(transforms=transforms_train, split='train')
test_dataset = Alpine(transforms=transforms_train, split='test')
val_dataset = Alpine(transforms=transforms_train, split='val')

# data loader
train_loader = DataLoader(train_dataset, 
                          batch_size  = 16, 
                          shuffle     = False, 
                          num_workers = 1,
                          pin_memory  = True)
test_loader = DataLoader(test_dataset, 
                          batch_size  = 16, 
                          shuffle     = False, 
                          num_workers = 1,
                          pin_memory  = True)
val_loader = DataLoader(val_dataset, 
                          batch_size  = 16, 
                          shuffle     = False, 
                          num_workers = 1,
                          pin_memory  = True)

####### COMPUTE MEAN / STD

# placeholders
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

#The first two steps are done in the snippet below. Note that we set axis = [0, 2, 3] 
#to compute mean values with respect to axis 1. The dimensions of inputs is [batch_size x 3 x image_size x image_size],
#so we need to make sure we aggregate values per each RGB channel separately.

# loop through images
for img,img_label in tqdm(train_loader):
    psum    += img.sum(axis        = [0, 2, 3])
    psum_sq += (img ** 2).sum(axis = [0, 2, 3])
    
for img,img_label in tqdm(test_loader):
    psum    += img.sum(axis        = [0, 2, 3])
    psum_sq += (img ** 2).sum(axis = [0, 2, 3])
    
for img,img_label in tqdm(val_loader):
    psum    += img.sum(axis        = [0, 2, 3])
    psum_sq += (img ** 2).sum(axis = [0, 2, 3])
    
####### FINAL CALCULATIONS

# image count
s = 0
for image in test_csv :
    s+=1
for image in train_csv :
    s+=1
for image in val_csv :
    s+=1
# pixel count
image_size = train_dataset[0][0].shape #[3,400,400]     
count = s * image_size[1] * image_size[1]

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

# output
print('mean: '  + str(total_mean))
print('std:  '  + str(total_std))


  0%|          | 0/460 [00:00<?, ?it/s]

NameError: name 'df' is not defined

In [84]:
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss()

# create 21 dummy predictions with 21 classes and random logits
"""CLASSES : rocks (1), scree (2), sparse rocks (3), water (4), glacier and
permanent snow (5), forest(6), sparse forest(7), grasslands and others (8)."""
num_classes = 8
batch_size = 16


In [ ]:
from torch.optim import SGD

learning_rate = 0.01
# TODO instantiate the SGD optimizer
optimizer = SGD(model.parameters(),lr=learning_rate)

In [ ]:
import torch.nn.functional as F

def training_step(batch, model, optimizer, device="cuda"):
    # TODO fill this function with the training step code
    model.train()
    optimizer.zero_grad()
    model.zero_grad()
    
    # TODO retrieve image and label from the batch
    x, y = batch
    
    # TODO move model and code to GPU
    model = model.to(device)
    x = x.to(device)
    y = y.to(device)
    
    # TODO forward pass
    y_hat =  model(x)
    
    # TODO loss calculation
    loss = criterion(y_hat,y)
  
    # TODO implement backprop and model update
     # backpropoagation of gradients
     # update model parameters
    loss.backward()
    optimizer.step()

    # lets also calculate accuracy for fun
    # FYI
    # .cpu() moves the data back to cpu (if on GPU)
    # .detach() removes gradients (we dont need them for accuracy)
    # .numpy() converts the tensor to numpy for better handling later
    predictions = y_hat.argmax(1).cpu().detach().numpy()
    ground_truth = y.cpu().detach().numpy()
    
    # accuracy is the mean of correct (1) and incorrect (0) classifications
    accuracy = (predictions == ground_truth).mean()
  
    return loss, accuracy